In [13]:
import numpy as np
import seaborn as sns
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import pandas as pd
import os
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
import argparse
import locale
from sklearn import tree
from sklearn.neighbors import KNeighborsRegressor



test_data = pd.read_csv('test.csv') 
train_data = pd.read_csv('train.csv') 


# input path for the images
base_path = "../IDS2021_HW09/"


#We select the necessary metadata and convert it into the necessary form
needed_data = train_data[['Id', 'Pawpularity']]
size = len(needed_data)
needed_data['Pawpularity'] = needed_data['Pawpularity'].astype(str)
i=0
for expression in os.listdir(base_path + "files/train/"):
    needed_data['Id'].iloc[i] = expression
    i+=1



#We load all the images into an array
images = []
for filename in needed_data['Id']:
    image = cv2.imread(base_path + "files/train/" + filename)
    image = cv2.resize(image, (64,64))
    images.append(image)

    
image_array = np.array(images)

    
#funtion to make the neural network
def make_cnn_model(image_width, image_height, image_depth, filters=(16, 32, 64), regress=False):
    Shape = (image_height, image_width, image_depth)
    inputs = Input(shape=Shape)
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(20)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(0.5)(x)
    x = Dense(8)(x)
    x = Activation("relu")(x)
    if regress:
        x = Dense(1, activation="linear")(x)
    model = Model(inputs, x)
    return model

image_array = image_array /255.0
split = train_test_split(train_data['Pawpularity'], image_array, test_size=0.25, random_state=40)
(train, test, trainImages, testImages) = split




<ipython-input-13-ca121638a718>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needed_data['Pawpularity'] = needed_data['Pawpularity'].astype(str)
C:\Users\uzingel\Anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [7]:
split = train_test_split(train_data['Pawpularity'], image_array, test_size=0.25, random_state=40)
(train, test, trainImages, testImages) = split

In [10]:
#We convert the pawpularity to be between [0,1]
maxPrice = train.max()
trainY = train / maxPrice
testY = test / maxPrice
#We make and train the model
model2 = make_cnn_model(64, 64, 3, regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model2.compile(loss="mean_absolute_percentage_error", optimizer=opt)

print("[INFO] training model...")
model2.fit(x=trainImages, y=trainY, 
    validation_data=(testImages, testY),
    epochs=200, batch_size=1024)

[INFO] training model...
Epoch 1/200
8/8 [==============================] - 29s 4s/step - loss: 616.1407 - val_loss: 141.7598
Epoch 2/200
8/8 [==============================] - 28s 4s/step - loss: 443.2402 - val_loss: 124.5618
Epoch 3/200
8/8 [==============================] - 28s 4s/step - loss: 368.3957 - val_loss: 105.0055
Epoch 4/200
8/8 [==============================] - 28s 4s/step - loss: 288.8895 - val_loss: 94.1747
Epoch 5/200
8/8 [==============================] - 29s 4s/step - loss: 220.0220 - val_loss: 90.8432
Epoch 6/200
8/8 [==============================] - 28s 4s/step - loss: 171.4921 - val_loss: 86.0182
Epoch 7/200
8/8 [==============================] - 28s 4s/step - loss: 144.7242 - val_loss: 76.2039
Epoch 8/200
8/8 [==============================] - 28s 4s/step - loss: 126.0893 - val_loss: 65.3121
Epoch 9/200
8/8 [==============================] - 28s 4s/step - loss: 111.5838 - val_loss: 59.8680
Epoch 10/200
8/8 [==============================] - 29s 4s/step - loss: 

8/8 [==============================] - 27s 3s/step - loss: 49.7041 - val_loss: 63.6092
Epoch 83/200
8/8 [==============================] - 27s 3s/step - loss: 48.3774 - val_loss: 63.6593
Epoch 84/200
8/8 [==============================] - 26s 3s/step - loss: 49.0980 - val_loss: 64.0486
Epoch 85/200
8/8 [==============================] - 26s 3s/step - loss: 48.7912 - val_loss: 64.2764
Epoch 86/200
8/8 [==============================] - 26s 3s/step - loss: 49.1345 - val_loss: 63.7229
Epoch 87/200
8/8 [==============================] - 26s 3s/step - loss: 48.4898 - val_loss: 61.1545
Epoch 88/200
8/8 [==============================] - 29s 4s/step - loss: 48.7382 - val_loss: 62.1817
Epoch 89/200
8/8 [==============================] - 26s 3s/step - loss: 49.3158 - val_loss: 62.3797
Epoch 90/200
8/8 [==============================] - 26s 3s/step - loss: 49.1814 - val_loss: 62.1885
Epoch 91/200
8/8 [==============================] - 26s 3s/step - loss: 48.2237 - val_loss: 62.1786
Epoch 92/200


8/8 [==============================] - 27s 3s/step - loss: 42.4356 - val_loss: 67.1805
Epoch 164/200
8/8 [==============================] - 26s 3s/step - loss: 42.8690 - val_loss: 64.9913
Epoch 165/200
8/8 [==============================] - 27s 3s/step - loss: 43.2433 - val_loss: 61.5701
Epoch 166/200
8/8 [==============================] - 29s 4s/step - loss: 43.3899 - val_loss: 62.1207
Epoch 167/200
8/8 [==============================] - 27s 3s/step - loss: 42.3612 - val_loss: 61.3799
Epoch 168/200
8/8 [==============================] - 27s 3s/step - loss: 41.9150 - val_loss: 62.8598
Epoch 169/200
8/8 [==============================] - 27s 3s/step - loss: 41.4972 - val_loss: 63.0483
Epoch 170/200
8/8 [==============================] - 27s 3s/step - loss: 42.9213 - val_loss: 63.6697
Epoch 171/200
8/8 [==============================] - 27s 3s/step - loss: 40.8523 - val_loss: 63.3599
Epoch 172/200
8/8 [==============================] - 27s 3s/step - loss: 41.7173 - val_loss: 65.7447
Epoc

In [11]:
#Predict the values of images
preds = model2.predict(testImages)

#Calculate the differences between results and the real values
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

#Calculate the mean of our prediction accuracy
mean = np.mean(absPercentDiff)
print(mean)
print(preds)
print(testY)

62.113909339476976
[[0.22978726]
 [0.283046  ]
 [0.30846784]
 ...
 [0.21248013]
 [0.29881638]
 [0.3078935 ]]
759     0.29
5719    0.38
7291    0.39
2643    0.84
4072    0.04
        ... 
6508    0.28
3026    0.30
1937    0.15
6421    0.29
6411    0.27
Name: Pawpularity, Length: 2478, dtype: float64


In [92]:
#Predict the values for the test data

needed_data_test = test_data
j = 0
for expression in os.listdir(base_path + "test/"):
    needed_data_test['Id'].iloc[j] = expression
    j+=1


images2 = []
for filename in needed_data_test['Id']:
    image2 = cv2.imread(base_path + "test/" + filename)
    image2 = cv2.resize(image2, (64, 64))
    images2.append(image2)

    
image_array2 = np.array(images2)
image_array2 = image_array2 /255.0

#Results
predict2 = predict2 * 100
predict2 = model2.predict(image_array2)

In [18]:
#Here we work with the metadata

#Split the data into correct segments
validation = train_data.iloc[0:1000]
training = train_data.iloc[1000:]
y_train = training['Pawpularity']
X_train = training.drop(columns=['Pawpularity', 'Id'])
X_test = validation.drop(columns=['Id', 'Pawpularity'])
y_test = validation['Pawpularity']


#DecisionTreeRegression model
model_reg1 = tree.DecisionTreeRegressor(random_state=1)
model_reg1.fit(X_train, y_train)
results1 = model_reg1.predict(X_test)
diff = results1 - y_test
percentDiff = (diff / y_test) * 100
absPercentDiff = np.abs(percentDiff)

#Calculate the mean of our prediction accuracy
mean_tree = np.mean(absPercentDiff)
print(mean_tree)

#First KNeighborsRegression model
model_reg2 = KNeighborsRegressor(n_neighbors = 1)
model_reg2.fit(X_train, y_train)
results2 = model_reg2.predict(X_test)
diff2 = results2 - y_test
percentDiff2 = (diff2 / y_test) * 100
absPercentDiff2 = np.abs(percentDiff2)

#Calculate the mean of our prediction accuracy
meanKN_1 = np.mean(absPercentDiff2)
print(meanKN_1)

#Second KNeighborsRegression model
model_reg3 = KNeighborsRegressor(n_neighbors = 15)
model_reg3.fit(X_train, y_train)
results3 = model_reg3.predict(X_test)
diff3 = results3 - y_test
percentDiff3 = (diff3 / y_test) * 100
absPercentDiff3 = np.abs(percentDiff3)

#Calculate the mean of our prediction accuracy
meanKN_2 = np.mean(absPercentDiff3)
print(meanKN_2)

87.39596637751156
82.79069680471966
92.9610992271269


In [29]:
#Create a neural network for metadata analysis

def model_for_metadata(dim, regress=False):
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))
    if regress:
        model.add(Dense(1, activation="linear"))
    return model

In [30]:
#Split the training data
(train1, test1) = train_test_split(train_data, test_size=0.25, random_state=42)

#Normalize the pawpularity
maxPrice_metadata = train1["Pawpularity"].max()
trainY_metadata = train1["Pawpularity"] / maxPrice_metadata
testY_metadata = test1["Pawpularity"] / maxPrice_metadata

#Drop unnecessary colmuns
trainX_metadata = train1.drop(columns=['Id', 'Pawpularity'])
testX_metadata = test1.drop(columns=['Id', 'Pawpularity'])

#Create and compile model
model_for_metadata = model_for_metadata(trainX_metadata.shape[1], regress=True)
model_for_metadata.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [31]:
#Train the model
model_for_metadata.fit(x=trainX_metadata, y=trainY_metadata, 
    validation_data=(testX_metadata, testY_metadata),
    epochs=100, batch_size=1024)

Epoch 1/100
8/8 [==============================] - 0s 41ms/step - loss: 93.0935 - val_loss: 69.5617
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 63.6535 - val_loss: 61.0770
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 61.0941 - val_loss: 58.7368
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 59.0236 - val_loss: 57.7553
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 58.5506 - val_loss: 57.5685
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 58.2472 - val_loss: 57.3485
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 57.9812 - val_loss: 57.2992
Epoch 8/100
8/8 [==============================] - 0s 5ms/step - loss: 57.8741 - val_loss: 57.2319
Epoch 9/100
8/8 [==============================] - 0s 6ms/step - loss: 57.8007 - val_loss: 57.1787
Epoch 10/100
8/8 [==============================] - 0s 6ms/step - loss: 57.7472 - val_loss: 57.1442
Epoch 11

Epoch 83/100
8/8 [==============================] - 0s 11ms/step - loss: 56.8922 - val_loss: 56.8241
Epoch 84/100
8/8 [==============================] - 0s 7ms/step - loss: 56.8834 - val_loss: 56.8340
Epoch 85/100
8/8 [==============================] - 0s 9ms/step - loss: 56.8687 - val_loss: 56.8202
Epoch 86/100
8/8 [==============================] - 0s 11ms/step - loss: 56.8600 - val_loss: 56.8117
Epoch 87/100
8/8 [==============================] - 0s 11ms/step - loss: 56.8587 - val_loss: 56.8063
Epoch 88/100
8/8 [==============================] - 0s 10ms/step - loss: 56.8509 - val_loss: 56.8013
Epoch 89/100
8/8 [==============================] - 0s 10ms/step - loss: 56.8456 - val_loss: 56.7907
Epoch 90/100
8/8 [==============================] - 0s 11ms/step - loss: 56.8424 - val_loss: 56.7916
Epoch 91/100
8/8 [==============================] - 0s 10ms/step - loss: 56.8398 - val_loss: 56.7963
Epoch 92/100
8/8 [==============================] - 0s 11ms/step - loss: 56.8412 - val_loss: 

In [32]:
#Analyze results

#Predict scores
preds_for_metadata = model_for_metadata.predict(testX_metadata)

#Calculate the differences between results and the real values
diff_for_metadata = preds_for_metadata.flatten() - testY_metadata
percentDiff_for_metadata = (diff_for_metadata / testY_metadata) * 100
absPercentDiff_for_metadata = np.abs(percentDiff_for_metadata)

#Calculate the mean of our prediction accuracy
mean_metadata = np.mean(absPercentDiff_for_metadata)
print(mean_metadata)

56.800626150214576
